# Generate Stock Time Series Data

In [38]:
import numpy as np
import pandas as pd
import bokeh.plotting as bk
import datetime as dt
# hover pan, zoom, reset, lables, etc tools
from bokeh.models import HoverTool, Label, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool
import requests

API_URL = 'https://api.iextrading.com/1.0'

# Make request to API and get data

In [2]:
res = requests.get(f'{API_URL}/stock/msft/chart/5y')

In [3]:
data = res.json()

In [4]:
df = pd.DataFrame(data)

In [5]:
df.sample(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
1170,-2.28130,2.359886,-2.400,92.7594,2018-04-30,95.6124,"Apr 30, 18",92.3924,95.5465,41523605,41523605,93.5665
147,-0.06281,0.293503,-0.176,35.7109,2014-04-07,36.1326,"Apr 7, 14",35.6571,35.8545,37559615,37559615,35.8798


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 12 columns):
change              1259 non-null float64
changeOverTime      1259 non-null float64
changePercent       1259 non-null float64
close               1259 non-null float64
date                1259 non-null object
high                1259 non-null float64
label               1259 non-null object
low                 1259 non-null float64
open                1259 non-null float64
unadjustedVolume    1259 non-null int64
volume              1259 non-null int64
vwap                1259 non-null float64
dtypes: float64(8), int64(2), object(2)
memory usage: 118.1+ KB


# Chart No 1 - Candlestick Chart Setup

In [7]:
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

### CALCULATE THE PERCENTAGE OF CHANGE

In [8]:
df['changePercent'] = df['changePercent'].apply(lambda x: str(x) +'%')

In [9]:
# axis of one means go down the comumns, not the whole row. Default of 0 operates on the rows.
df['mid'] = df.apply(lambda x: (x['open'] + x['close'])/2, axis=1)
df.head(1)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs,mid
0,0.035355,0.0,0.128%,27.6079,2013-09-05,27.7891,"Sep 5, 13",27.356,27.4886,71623991,71623991,27.6561,0,27.54825


In [10]:
df['height'] = df.apply(
    lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.001, 
    axis=1)

In [11]:
df.sample(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs,mid,height
383,0.165865,0.387165,0.435%,38.2967,2015-03-16,38.3704,"Mar 16, 15",38.0386,38.2137,35273452,35273452,38.2056,383,38.25520,0.0830
130,-0.340961,0.231430,-0.993%,33.9972,2014-03-13,34.4996,"Mar 13, 14",33.7741,34.4727,32169650,32169650,34.5533,130,34.23495,-0.4755


In [12]:
inc = df.close > df.open
dec = df.close < df.open
w = .3

In [13]:
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

### Define type of tools we are going to use

In [14]:
hover = HoverTool(
    tooltips=[
        ('date', '@date'),
        ('low', '@low'),
        ('high', '@high'),
        ('open', '@open'),
        ('close', '@close'),
        ('percent', '@percent'),
    ]
)

In [15]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

### Present the data

In [16]:
p = bk.figure(plot_width=1500, plot_height=800, tools=TOOLS, title='Microsoft', toolbar_location='above')

### Put the data into the chart

In [17]:
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha=w

In [18]:
descriptor = Label(x=70, y=70, text='Label goes here')
# adds label onto the chart
p.add_layout(descriptor)

In [19]:
# set up your outer tails, and then the rectangles that go inside of the tails
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color='green')

bokeh.models.renderers.GlyphRenderer(
    id='1494c6b9-adc9-4f02-9941-f7f41871ebb7',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='f43df8b6-66cb-4625-90b2-11277cdcd918',
        callback=None,
        data={'y0': {'__ndarray__': 'C0YldQLKO0Dwp8ZLNxk8QPrt68A5ozxACRueXikbPUDtnjws1Do9QC/dJAaBhT1Avp8aL93EPUBO0ZFc/iM9QJLLf0i//TxAXrpJDAIrPUAUP8bctdQ9QP7UeOkmcT1AaQBvgQS1PUCjAbwFEhQ+QF66SQwCCz5A+MJkqmD0PUB3LSEf9Cw+QMrDQq1pfj5AZMxdS8jXPkCTqYJRSe0+QJOpglFJ7T5AzczMzMwcP0D8qfHSTaI/QJOpglFJzT9AUwWjkjo5QECjAbwFEuRAQJhMFYxKskBAk6mCUUndQECL/WX35NlAQPH0SlmGqEBAke18PzW2QEABTYQNT8dAQNUJaCJs0EBAW0I+6NkMQUCx4emVskRBQM/3U+OlI0FAJJf/kH5bQUCCc0aU9k5BQG8Sg8DKeUBAh6dXyjJMQEA2zTtO0XFAQBNhw9MrjUBAKA8LtaaxQEAijnVxG6VAQJZDi2znu0BAgSbChqc/QEA3iUFg5RhAQE5iEFg59D9AYOXQItthQEBvEoPAynlAQBUdyeU/NEBAzhlR2htsQEDOGVHaG2xAQBB6Nqs+30BAo5I6AU0kQEDsL7snD0tAQPtcbcX+YkBA5fIf0m+XQEDKMsSxLr5AQEATYcPT20BAbVZ9rrbSQEBkO99Pje9AQCEf9GxW/UBAsVBrmnc0QUAdyeU/pP9AQHBfB84ZQUFACKwcWmRDQUDyQc9m1SdBQPRsVn2uJkFA2T15WKg9QUCyne+nxhtBQN6Th4VaO0FAH/RsVn3mQUDcaABvgTxCQF3cRgN4O0JA3nGKjuQ6QkC8BRIUP55CQH9qvHSTqEJAtRX7y+7pQUCad5yiIzFCQGDl0CLb2UFAke18PzWuQUBMN4lBYO1BQGFUUiegIUJANV66SQwCQkBKDAIrh+5BQNlfdk8ehkJAUiegibC5QUAf9GxWfeZBQHh6pSxD9EFA6gQ0ETZMQkAdWmQ73/9BQL1SliGO/UFAtFn1udo6QkC0WfW52jpCQMP1KFyPgkJAE2HD0ys9QkBt5/up8aJCQFRSJ6CJmEJAVn2utmLHQkAxmSoYlcxCQDqSy39I70JACD2bVZ/bQkAg0m9fB+ZCQFInoImw+UJADwu1pnn/QkBeS8gHPRtDQInS3uALC0NAPL1SliHuQkAr9pfdkwdDQFInoImw+UJAzhlR2hsEQ0COdXEbDTBDQL6fGi/dBERAtaZ5xylCREAep+hILmdEQIV80LNZLURAwcqhRbajQ0Bhw9MrZaFDQDY8vVKWsUNAS1mGONb1Q0C3Yn/ZPRFEQPH0SlmGSERAYcPTK2VhREA/V1uxv5xEQDXvOEVHkkRAtFn1udpyRECq8dJNYqhEQESLbOf7gURA+FPjpZuUREAofoy5a+FEQJEPejarRkVApgpGJXVaRUCCc0aU9lZFQGdEaW/wXUVAFK5H4XpMRUDEQq1p3jlFQLWmeccpSkVAjgbwFkigRUCYbhKDwGpFQBniWBe3MUVANBE2PL0qRUAUrkfhegxFQJp3nKIjUUVA0m9fB865Q0DLEMe6uJVDQFMFo5I6+UNADAIrhxYRREDRkVz+Q3JEQIy5awn5qERA2ht8YTL1REDr4jYawCNFQCV1ApoIO0VApgpGJXVaRUDBOSNKe5NFQELPZtXnskVA9GxWfa7GRUDKw0KtaTZGQA6+MJkqWEZAdCSX/5A+RkAT8kHPZj1GQAu1pnnHkUZA8IXJVMGYRkAB3gIJikdGQFwgQfFj9EVAm+Ydp+hQRkC2hHzQs3FGQIenV8oy7EVALNSa5h3XRUBnRGlv8NVFQIZa07zjBEVAhXzQs1m9RUB+HThnRAFGQKTfvg6cA0ZAwaikTkBTRkDEsS5uo7FFQIxK6gQ0QUVAKqkT0ETYRUAydy0hHyxFQEhQ/BhzV0VAx0s3iUGQRUDjpZvEIJBFQE0VjErqRENAW9O84xTtQkA2zTtO0YFDQPaX3ZOHjUNASgwCK4e2Q0DEQq1p3hFEQD0s1JrmDURAVg4tsp03REDA7J48LFxEQKrx0k1iUERAN4lBYOVgREDA7J48LFxEQOPHmLuW6ENAMnctIR/sQ0B0JJf/kN5DQN6Th4VaG0NAHThnRGkvQ0Dn+6nx0kVDQE9AE2HDu0NArK3YX3afQ0AFxY8xd81DQGsr9pfd40NAVg4tsp3HQkCrz9VW7D9DQDC7Jw8LLUNAhxbZzvdTQ0DF/rJ78mBDQOOlm8QgkENA4C2QoPiBQ0BrK/aX3eNDQFfsL7sn30NAAG+BBMUXRECuR+F6FC5GQOm3rwPnLEZAPzVeukmsRkBVMCqpE7hGQAYSFD/GhEZAxLEubqOxRUCXkA96NhtGQI/k8h/S90VA5x2n6Eh+RkA6I0p7gzdGQHlYqDXNK0ZANV66SQw6RkB7FK5H4RJGQDsBTYQNJ0ZAOwFNhA0nRkCsrdhfdrdFQKVOQBNhK0VAKH6MuWtxRUAFNBE2PF1FQJZDi2zns0VAxty1hHygRECP5PIf0idFQHEbDeAtUEVAHVpkO99HRUACK4cW2aZFQERpb/CFsUVAUI2XbhLbRUCJQWDl0PJFQBfZzvdTw0VAAJF++zpwRUClLEMc63pFQERpb/CFsUVAQmDl0CL7RUAOLbKd7x9GQERpb/CFsUVAhJ7Nqs8FRkAAb4EExb9FQDojSnuD10VARUdy+Q8BRkBAE2HD0yNGQHNoke18N0RAZapgVFL/Q0CHFtnO94NEQGsr9pfdm0RAVp+rrdg/REBkO99PjYdEQBlz1xLyiURAqRPQRNhwREDF/rJ78lhEQJT2Bl+YrERALGUZ4li3REAnMQisHMJEQEdy+Q/pj0RAtvP91HiZREARx7q4jVZEQL3jFB3JrURAvVKWIY5FRUDswDkjSuNFQIV80LNZBUZAklz+Q/otRkB/+zpwzvhFQNXnaiv2/0VAXCBB8WP8RUCMuWsJ+TBGQJqZmZmZWUZAUwWjkjpRRkBTliGOddlGQPcGX5hMPUlACRueXilbSUD7OnDOiDJJQJzEILBy6EhAT6+UZYhjSUASpb3BF55JQNIA3gIJqklAZMxdS8gnSUC8dJMYBF5JQOjZrPpclUlANV66SQxqSUC/fR04Z/RJQBQ/xty1bEpA2qz6XG1NSkD3deCcEV1KQKfoSC7/8UlA5q4l5INuSkBtxf6yexJKQO0NvjCZQkpAwhcmUwVLSkCe76fGS7dKQDj4wmSqwElAgSbChqcHSkBcj8L1KPRIQOC+DpwzGklABTQRNjwlSECdgCbChpdIQD9XW7G/pEhADAIrhxaJSEAWak3zjuNJQAdfmEwVnEdA6pWyDHGMR0AbL90kBtFHQLyWkA96LkhA0gDeAgn6SEA9LNSa5hVJQKpgVFInYEhADXGsi9uoSEBOYhBYOeRIQEVHcvkPEUlAeAskKH6sSEAkKH6MuQNJQAMJih9jHklAr5RliGNdSUCvlGWIY11JQABvgQTF10lAY+5aQj4ISkB0JJf/kIZJQOjZrPpcrUlAnl4pyxC3SUA6kst/SPdJQL3jFB3JVUpA1lbsL7tPSkBSSZ2AJlJKQGN/2T15IEpAhA1Pr5TtSUBoke18Pz1KQKCJsOHpTUpAcvkP6bd3SkA3iUFg5chKQHgLJCh+rEhA/7J78rDIR0AwTKYKRgVIQO/Jw0KtsUdASZ2AJsLOR0AoDwu1ptlHQKs+V1uxL0hAyxDHuriFSEDkFB3J5ZdIQB1aZDvfX0hAxLEub

In [20]:
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color='red')

bokeh.models.renderers.GlyphRenderer(
    id='3fa8f4c5-128b-4018-b3b4-b6f8cb7538f7',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='ed05d4d5-0101-4aba-bcff-c788f5b8f774',
        callback=None,
        data={'y0': {'__ndarray__': 'WMoyxLG+O0Dwp8ZLN/k8QJXUCWginD1AhXzQs1mVPUDysFBrmpc9QOhqK/aXDT1AzO7Jw0INPkDP91Pjpcs9QNk9eViodT1Ak6mCUUntPkBn1edqKwY/QJXUCWgifD5AMuauJeQjPkAZ4lgXtwlAQBQ/xty1lD9ApgpGJXWSP0CWsgxxrIs/QJayDHGsiz9AB/AWSFDMQEB9PzVeurFAQOjZrPpcnUBAPQrXo3DNQEAyVTAqqZtAQLIubqMBlEBAEqW9wRfmQEAy5q4l5MNAQJ2AJsKGT0FAbcX+snsqQUBR2ht8YVJBQPW52or9DUFAmG4Sg8DCQEC+MJkqGK1AQHWTGARWhkBAH/RsVn1GQEBoke18P21AQGN/2T15wEBAVn2utmKnQEAXt9EA3pJAQGiR7Xw/bUBAnRGlvcEXQECz6nO1Ffs/QPwYc9cSCkBAysNCrWlmQEAwTKYKRmVAQNzXgXNGLEBAApoIG54WQEDG3LWEfLhAQGiR7Xw/bUBA/Bhz1xLqQEDIBz2bVY9AQEw3iUFgPUBAXrpJDALzQECJ0t7gC/tAQMrDQq1p7kBAMzMzMzMbQUAlBoGVQytBQK8l5IOeNUFAdk8eFmoNQUByio7k8j9BQOcdp+hIvkFABOeMKO1dQkCWsgxxrGNCQAmKH2PuQkJASgwCK4fuQUCqglFJnbBCQNlfdk8ehkJA/fZ14JyRQkCMuWsJ+RBCQPCnxks3QUJAMzMzMzMDQkDiWBe30fBBQPLSTWIQQEJAGCZTBaN6QkCze/KwUBtCQLfRAN4CGUJAqoJRSZ3IQUBlqmBUUqdBQC0hH/RsRkJAcT0K16NAQkC0WfW52jpCQBNhw9MrPUJAnMQgsHIwQkB/2T15WChCQJzEILBykEJAGy/dJAZhQkC9UpYhjr1CQDy9UpYhjkJAqmBUUid4QkBlGeJYF99CQJhuEoPA8kJAmG4Sg8DyQkBEi2zn+xFDQFRSJ6CJ+EJAUiegibD5QkBUUiegifhCQJLLf0i/LUNAdLUV+8umREC8dJMYBGZEQMZtNIC3iERA6SYxCKxUREBO0ZFc/htEQM9m1edq60NAMZkqGJXsQ0CwA+eMKL1DQNGRXP5DikNAw/UoXI+iQ0BvEoPAypFDQGHD0ytloUNAiIVa07yjREDOiNLe4KtEQKrx0k1iqERApb3BFyajREB3vp8aL3VEQG1Wfa62qkRAKqkT0ERgRUCHp1fKMjxFQLWmeccpikVApN++DpxbRUDVCWgibGhFQFR0JJf/IEVAEHo2qz4nRUDaG3xhMvVEQBSuR+F6DEVAKH6MuWvhRECRD3o2q0ZFQJ2AJsKG90RA4C2QoPhBREBpAG+BBC1EQL7BFyZTfURA/Knx0k1SRUDXEvJBzw5FQBPyQc9mPUZAuB6F61HARkCNKO0NvpBGQJJc/kP6TUZAVFInoIlwRkC9UpYhjvVFQDVeukkM8kVAv30dOGcMRkBUUiegiXBGQIBIv30dMEZAukkMAitnRkARx7q4jR5GQJLLf0i/zUVA9P3UeOnORUClvcEXJjNFQI2XbhKDQEZAhA1Pr5QlRkB5WKg1zdtFQEVHcvkPyUVA63O1FfuzRUDiWBe30WBFQInS3uALY0VAxty1hHzgRUCrPldbsb9FQMIXJlMF60VA5fIf0m8nRUDyQc9m1TdFQCQofoy5G0VA2V92Tx6uRUBsCfmgZ8NDQAisHFpkk0NAofgx5q4FQ0CRD3o2qy5DQKyt2F92T0NACKwcWmSTQ0Di6ZWyDJFDQPtcbcX+gkNAWYY41sVFREBR2ht8YSJEQP7UeOkmaURA6Nms+lxdREA3GsBbIDFEQM3MzMzM3ENAryXkg56tQ0CTOgFNhIVDQOPHmLuWMENAV+wvuyffQ0Dl0CLb+cZDQGsr9pfdK0NAylTBqKQWQ0Ck374OnCNDQKyt2F9250JAaJHtfD/FQkBz1xLyQU9DQFpkO99PNUNAdk8eFmpdQ0AzMzMzMztDQOELk6mC4UNAl5APejbTRkAjSnuDL4RGQDhnRGlvMEZA30+Nl24CRkBt5/up8RJGQLN78rBQQ0ZAVcGopE6IRkDaG3xhMvVFQBPyQc9mvUVA2/l+arxERkABTYQNTw9GQNobfGEy9UVAQs9m1eciRkDKVMGopN5FQN/gC5OpkkVAOUVHcvmHRUCEns2qz01FQFJJnYAmwkVAeJyiI7mMRUDJ5T+k37ZFQBiV1AloqkVAZ9Xnait2RUALRiV1AnJFQECk374OXEVAZ9Xnait2RUBEaW/whflEQNuK/WX3vERARGlv8IX5REBUUiegicBEQIts5/upoURAJ8KGp1fSREAcfGEyVfhEQJ/Nqs/V7kRA2V92Tx5WRUD8GHPXEipFQMiYu5aQ90VAz/dT46XLRUAldQKaCHNGQDsBTYQNJ0ZAP8bctYTERUA6I0p7g9dFQKfoSC7/+UVAcF8HzhmxRUAm5IOezTpFQCzUmuYdL0RA097gC5OBREDvOEVHcuFDQEi/fR04F0RA4JwRpb25REB0RpT2BkdEQO84RUdyAUVAjSjtDb6IREAgY+5aQp5EQMWPMXct4URAarx0kxiUREB6xyk6kuNEQNGRXP5DGkZAjLlrCfkwRkDmP6TfvmZGQJp3nKIjYUlAqoJRSZ0gSUCIhVrTvDNJQH9qvHSTiElAhlrTvOOcSUCxUGuad0RJQJSHhVrTTElA30+Nl24ySUC5jQbwFuBIQIC3QILiJ0lA8kHPZtWvSUA/xty1hIRJQABvgQTFl0lAarx0kxh8SUD5oGez6tNJQE5iEFg5TEpA6Nms+lw1SkBhw9MrZUFKQJ2AJsKGJ0pAorQ3+MLkSUAFo5I6AUVKQFUwKqkTsEpAfoy5awlRSkBQ/Bhz1wJKQE5iEFg5TEpAyeU/pN+uSkDkFB3J5WdKQBsN4C2QkElACmgibHgiSUDSb18HzglJQK5H4XoU1khAqRPQRNhoSUDqBDQRNmxIQAyTqYJRSUhAk6mCUUk9SEC8dJMYBL5IQIC3QILih0hAFmpN847jSUAPnDOitH9JQNejcD0KF0lAfh04Z0TRSECP5PIf0m9IQME5I0p7S0dAPL1SliGuR0BWn6ut2A9JQJEPejarvkhA097gC5PJSECQMXctIe9IQMNkqmBUckhA0ZFc/kMSSUAlBoGVQ9NIQNxoAG+BhEhAZvfkYaEOSUD4U+OlmwRKQFyPwvUorElAQxzr4jaySUCU9gZfmCxKQCEf9GxW/UlAtaZ5xykqSkB88rBQaxpKQLsnDwu13kpA9bnaiv29SkAwTKYKRp1KQH/7OnDO0EhA845TdCTHSECqYFRSJ2BIQA+cM6K0B0hAB1+YTBXcR0DhehSuR/FHQGB2Tx4WgkhAZF3cRgOQSEC5/If026dIQNUJaCJsIEhAObTIdr4nSEDnHafoSMZIQBdIUPwYI0lAQKTfvg78SEAnoImw4SFJQH/7OnDOyEhA8BZIUPzgR0DPZtXnauNHQLdif9k9CUhANV66SQw6SEBXW7G/7IZIQCJseHqlbEdAUwWjkjqZSEBPQBNhw6tJQAg9m1Wfu0lA9+Rhoda8SUBxPQrXo7BJQI/C9ShcF0tAFNBE2PDMSkBtxf6yexJLQPjCZKpgDEtA9GxWfa4eS0DAWyBB8ctLQAdfmEwVvEtAih9j7lqiS0DJdr6fGudLQB4Wak3zJkxAQj7o2awaTEAcfGEyVehLQP8h/fZ1uEtA/yH99

In [21]:
# bk.show(p) 
# SHOWS CHART inside notebook

In [22]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)

bokeh.models.renderers.GlyphRenderer(
    id='02c7214a-75eb-4a3a-9e05-ad0f4a8a65de',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='402dd9cc-4e66-410b-81f5-6aa0854e3d8a',
        callback=None,
        data={'changePercent': ['0.128%',
          '1.615%',
          '2.322%',
          '1.081%',
          '1.04%',
          '1.184%',
          '0.96%',
          '-0.156%',
          '0.154%',
          '0.815%',
          '1.526%',
          '0.03%',
          '0.901%',
          '1.013%',
          '0.059%',
          '2.086%',
          '1.096%',
          '0.938%',
          '0.435%',
          '0.808%',
          '0.115%',
          '0.086%',
          '0.056%',
          '1.168%',
          '1.948%',
          '4.203%',
          '0.747%',
          '2.128%',
          '-0.351%',
          '0.925%',
          '0.863%',
          '0.455%',
          '0.669%',
          '1.41%',
          '0.839%',
          '-0.364%',
          '1.644%',
          '0.899%',
          '0.531%',
          '0.164%',
          '1.517%',
          '1.256%',
          '0.971%',
          '0.0%',
          '0.322%',
          '0.775%',
          '1.435%',
          '2.287%',
          '2.739%',
          '0.354%',
          '0.666%',
          '1.075%',
          '0.546%',
          '2.659%',
          '1.005%',
          '1.05%',
          '0.656%',
          '1.019%',
          '0.794%',
          '0.374%',
          '0.027%',
          '0.241%',
          '0.64%',
          '0.609%',
          '1.041%',
          '1.189%',
          '1.668%',
          '0.105%',
          '0.529%',
          '0.658%',
          '-0.501%',
          '0.928%',
          '3.942%',
          '2.699%',
          '0.847%',
          '2.388%',
          '1.712%',
          '1.049%',
          '0.05%',
          '1.632%',
          '-0.384%',
          '-0.074%',
          '1.455%',
          '1.635%',
          '0.125%',
          '0.428%',
          '2.405%',
          '0.934%',
          '0.545%',
          '1.088%',
          '1.839%',
          '0.581%',
          '-0.201%',
          '1.689%',
          '0.825%',
          '1.487%',
          '0.074%',
          '2.207%',
          '-0.388%',
          '1.602%',
          '0.655%',
          '0.434%',
          '-0.072%',
          '0.41%',
          '0.744%',
          '0.683%',
          '1.27%',
          '0.408%',
          '0.072%',
          '0.455%',
          '0.036%',
          '0.972%',
          '0.736%',
          '3.84%',
          '0.359%',
          '0.324%',
          '0.225%',
          '1.19%',
          '1.146%',
          '0.741%',
          '1.287%',
          '0.431%',
          '1.175%',
          '0.714%',
          '1.115%',
          '0.601%',
          '0.022%',
          '1.225%',
          '-0.288%',
          '0.667%',
          '1.436%',
          '1.22%',
          '0.624%',
          '0.171%',
          '0.342%',
          '1.125%',
          '-0.513%',
          '0.344%',
          '1.799%',
          '1.117%',
          '0.804%',
          '0.065%',
          '0.721%',
          '2.745%',
          '-1.166%',
          '-1.111%',
          '2.082%',
          '1.031%',
          '1.815%',
          '1.442%',
          '-0.477%',
          '1.263%',
          '0.28%',
          '1.954%',
          '1.044%',
          '0.274%',
          '0.61%',
          '1.755%',
          '0.431%',
          '-0.041%',
          '-0.184%',
          '1.702%',
          '-0.242%',
          '0.995%',
          '0.59%',
          '1.694%',
          '1.581%',
          '-0.22%',
          '0.576%',
          '-0.466%',
          '1.284%',
          '3.892%',
          '0.295%',
          '0.671%',
          '0.98%',
          '0.667%',
          '1.271%',
          '2.942%',
          '1.671%',
          '0.324%',
          '2.635%',
          '-0.36%',
          '1.991%',
          '2.178%',
          '1.458%',
          '-0.118%',
          '1.675%',
          '1.81%',
    

In [23]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

bokeh.models.renderers.GlyphRenderer(
    id='917ece1e-b934-426f-9191-1095fd5b3768',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='4f69be43-947c-4a81-87b8-9f26518f38c5',
        callback=None,
        data={'changePercent': ['-0.266%',
          '-0.153%',
          '-0.693%',
          '0.393%',
          '-2.524%',
          '-0.87%',
          '-0.177%',
          '-1.712%',
          '-0.871%',
          '0.116%',
          '-1.172%',
          '-2.371%',
          '-0.118%',
          '5.961%',
          '-0.448%',
          '-0.141%',
          '-0.38%',
          '0.339%',
          '-1.781%',
          '-0.503%',
          '-0.612%',
          '-0.473%',
          '-1.694%',
          '-0.488%',
          '0.186%',
          '-0.77%',
          '-2.414%',
          '0.947%',
          '-1.537%',
          '-1.312%',
          '-1.037%',
          '-1.424%',
          '-0.99%',
          '-0.902%',
          '-0.489%',
          '-0.401%',
          '-0.668%',
          '-0.673%',
          '-2.113%',
          '-1.785%',
          '-0.643%',
          '-2.941%',
          '-1.382%',
          '-0.577%',
          '-0.664%',
          '0.348%',
          '2.08%',
          '-2.106%',
          '-3.594%',
          '-0.356%',
          '-1.458%',
          '0.214%',
          '-0.398%',
          '-0.186%',
          '-1.383%',
          '-0.781%',
          '-0.655%',
          '-0.211%',
          '-0.993%',
          '-0.708%',
          '-0.422%',
          '-0.395%',
          '-1.363%',
          '-1.081%',
          '-0.169%',
          '-0.822%',
          '-2.78%',
          '-0.176%',
          '-2.743%',
          '-0.175%',
          '-0.75%',
          '0.125%',
          '-0.881%',
          '-0.99%',
          '-0.775%',
          '-0.655%',
          '-0.938%',
          '-0.445%',
          '-1.59%',
          '-0.62%',
          '0.05%',
          '0.174%',
          '-0.448%',
          '-0.366%',
          '-1.226%',
          '-0.506%',
          '-0.608%',
          '-0.685%',
          '-0.336%',
          '-0.583%',
          '-0.738%',
          '-1.302%',
          '-0.239%',
          '-0.5%',
          '-0.263%',
          '0.119%',
          '1.021%',
          '-0.011%',
          '0.089%',
          '-1.047%',
          '-1.191%',
          '-0.193%',
          '-0.698%',
          '-0.96%',
          '-0.695%',
          '-0.669%',
          '-0.069%',
          '0.0%',
          '-0.838%',
          '-0.155%',
          '0.044%',
          '-0.365%',
          '-0.3%',
          '-0.748%',
          '-0.649%',
          '-0.974%',
          '-0.968%',
          '-1.062%',
          '-2.209%',
          '-0.172%',
          '-0.992%',
          '-0.305%',
          '0.0%',
          '-1.215%',
          '-1.988%',
          '-3.969%',
          '-0.863%',
          '0.183%',
          '-1.114%',
          '2.466%',
          '-1.223%',
          '-0.041%',
          '-0.06%',
          '-0.834%',
          '-1.067%',
          '-1.478%',
          '-0.813%',
          '-0.252%',
          '0.126%',
          '-0.329%',
          '-0.784%',
          '-0.86%',
          '-1.497%',
          '-1.45%',
          '-0.596%',
          '-3.235%',
          '-0.64%',
          '-0.54%',
          '-0.898%',
          '-0.906%',
          '-1.212%',
          '-0.93%',
          '-1.457%',
          '-0.841%',
          '-1.25%',
          '-0.526%',
          '-0.863%',
          '-1.034%',
          '-1.013%',
          '0.106%',
          '-9.253%',
          '-3.446%',
          '-3.832%',
          '0.775%',
          '0.577%',
          '-0.094%',
          '0.567%',
          '-0.516%',
          '0.046%',
          '-0.115%',
          '-0.136%',
          '-0.465%',
          '-1.367%',
          '-1.74%',
          '-1.914%',
          '-0.119%',
          '-2.287%',
          '-0.058%',
          '-3.357%',
          '-0.603%',
          '-0.582%',
          '-0.024

In [24]:
# How we'll save out to our file, basically a file.write. Use f string to give dynamic, title is the html doc
bk.save(p, './stocks_api_project/static/candle_stick.html', title='5yr_candlestick')

/Users/madelinepet/.local/share/virtualenvs/stocks_api_project-ZUhAC5B9/lib/python3.7/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/Users/madelinepet/codefellows/stocks_api_project/stocks_api_project/static/candle_stick.html'

# Chart No 2 - avg daily low versus year

### Format the date as a date object

In [25]:
df['date'] = pd.to_datetime(df['date'])

In [26]:
df['MONTH'] = df['date'].dt.month

In [27]:
df['DAY'] = df['date'].dt.day

In [28]:
df['YEAR'] = df['date'].dt.year

In [29]:
df.sample(3)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs,mid,height,MONTH,DAY,YEAR
164,-0.358905,0.300005,-0.99%,35.8904,2014-05-01,36.2134,"May 1, 14",35.8455,36.1058,28791455,28791455,35.9532,164,35.9981,-0.2154,5,1,2014
662,-3.786500,0.775445,-7.171%,49.0163,2016-04-22,49.6313,"Apr 22, 16",48.0602,49.1393,126834091,126834091,48.9466,662,49.0778,-0.1230,4,22,2016
260,-0.218215,0.532080,-0.513%,42.2975,2014-09-17,42.4521,"Sep 17, 14",42.0338,42.0611,38311924,38311924,42.2847,260,42.1793,0.2364,9,17,2014


### Split by Year and average the daily low per year

In [30]:
average_low_per_year = df[['low']].groupby(df.YEAR).mean()

In [31]:
average_low_per_year

,low
YEAR,
2013,31.207449
2014,38.040122
2015,42.919751
2016,52.237244
2017,69.893075
2018,96.005195


In [40]:
plot = bk.figure(plot_width=400, plot_height=400)
plot.vbar(x=df.YEAR, width=0.5, bottom=0,
       top=df.low, color="firebrick")

bk.save(plot, './stocks_api_project/static/year_versus_low.html', title='5yr_low_bar')


/Users/madelinepet/.local/share/virtualenvs/stocks_api_project-ZUhAC5B9/lib/python3.7/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/Users/madelinepet/codefellows/stocks_api_project/stocks_api_project/static/year_versus_low.html'

# Chart No 3- Daily Volitility- Average daily change versus day of month

### Separate by day of month

In [33]:
average_change_day_month = df[['change']].groupby(df.DAY).mean()
average_change_day_month

,change
DAY,
1,0.090621
2,-0.024480
3,-0.112170
4,0.093334
5,0.047547
6,0.104880
7,0.001286
8,0.077518
9,0.084259


In [36]:
plot_2 = bk.figure(plot_width=400, plot_height=400)
plot_2.vbar(x=df.DAY, width=0.5, bottom=0,
       top=df.change, color="blue")

bk.save(plot_2, './stocks_api_project/static/volatility.html', title='5yr_volitility_bar')

/Users/madelinepet/.local/share/virtualenvs/stocks_api_project-ZUhAC5B9/lib/python3.7/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/Users/madelinepet/codefellows/stocks_api_project/stocks_api_project/static/volatility.html'